In [22]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
from RBM import *
from principal_DNN_mnist import *
from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
from keras.datasets import mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Reshape data to fit input shape of model
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])

# Turn images to binary images
X_train = X_train > 127
X_test  = X_test > 127

#Original label to one hot coding label
label_test = np.zeros((Y_test.shape[0],10))
label_train = np.zeros((Y_train.shape[0],10))

for i in range(len(Y_train)):
    label_train[i, Y_train[i]] = 1

for i in range(len(Y_test)):
    label_test[i, Y_test[i]] = 1

Y_train = label_train
Y_test = label_test

# HERE TO CHOOSE SIZE OF TRAINING SET

Original size of MNIST is 60000 in training set and 10000 in test set.

Project asks us to test our model in training set of increasing size: 1000, 3000. 7000, 10000, 30000, 60000  

So choosing training_size to have these sizes: 1/60, 1/20, 7/60, 1/6, 1/2, 1.

In [24]:
training_size = 0.8

In [25]:
test_size = 0.8
# Data processing
train_idx = np.random.randint(0, X_train.shape[0], int(X_train.shape[0]*size))
test_idx = np.random.randint(0, X_test.shape[0], int(X_test.shape[0]*size))

X_train = X_train[train_idx]
Y_train = Y_train[train_idx]

X_test  = X_test[test_idx]
Y_test  = Y_test[test_idx]

#########

X_train = torch.from_numpy(X_train).to(device).float()
Y_train = torch.from_numpy(Y_train).to(device).float()
X_test = torch.from_numpy(X_test).to(device).float()
Y_test = torch.from_numpy(Y_test).to(device).float()

# HERE TO CHOOSE SIZE OF MODEL 

Size of model to be modify according to the project is number of neurons in each layer and number of layers.

In [26]:
hidden_layers = [150, 200] # To be modified. This is list of number of nodes in each layer (each value is a hidden layer) 

In [27]:
# Setting model hyper-parameters
nb_epochs_pretrain = 200
nb_epochs_scratch = 100
lr = 0.1
batch_size = 64
input_shape = X_train.shape[1]
nb_class = Y_train.shape[1]

###### Training method 1: Construct a DNN model. First train it unsupervisedly and then fine-tune with supervised backpropagation 

In [28]:
DNN_method_1 = DNN(input_shape, nb_class, hidden_layers)

In [29]:
DNN_method_1.pretrain_DNN(X_train, nb_epochs_pretrain, lr, batch_size)

Training layer 0
Training Started. 200 iterations.


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [02:07<00:00,  1.56it/s]


End Training
--------------
Training layer 1
Training Started. 200 iterations.


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [01:38<00:00,  2.03it/s]

End Training
--------------


In [30]:
DNN_method_1, loss = retropropagation(DNN_method_1, input = X_train, label = Y_train, epochs = nb_epochs_scratch, lr = lr, batch_size = batch_size)

100%|███████████████████████████████████████████████| 100/100 [00:33<00:00, 2.95it/s, loss =0.00006]


In [32]:
accuracy_method_1 = test_DNN(DNN_method_1, X_test, Y_test)

Accuracy: 0.97712


###### Training method 2: Construct a DNN model. Train it with supervised backpropagation from scratch

In [40]:
DNN_method_2 = DNN(input_shape, nb_class, hidden_layers)

In [41]:
DNN_method_2, loss = retropropagation(DNN_method_2, input = X_train, label = Y_train, epochs = nb_epochs_scratch, lr = lr, batch_size = batch_size)

100%|███████████████████████████████████████████████████| 100/100 [00:34<00:00, 2.88it/s, loss =nan]


In [42]:
accuracy_method_2 = test_DNN(DNN_method_2, X_test, Y_test)

Accuracy: 0.9625


### For each change of a hyperparameter of the model, we store accuracy_method_1 and accuracy_method_2 to plot these values in relation with chosen hyperparameter.

Note: For this implementation of pytorch in model, sometimes, the accuracy is 0. I don't know why after a lot of research on the code (it seems that there is problem with weight initialization). In that case, re-train the model and calculate the accuracy.